In [ ]:
import time

import masters.plot as plot
import masters.process as process
import masters.db as db

In [ ]:
%matplotlib widget

In [ ]:
fig, ax = plot.setup_figure()

In [ ]:
REDIS_HOST = !kubectl get secret redis-connection -o jsonpath='{.data.REDIS_HOST}' | base64 --decode
REDIS_PORT = !kubectl get secret redis-connection -o jsonpath='{.data.REDIS_PORT}' | base64 --decode
redis = db.create_client(REDIS_HOST[0], REDIS_PORT[0])

In [ ]:
workflow = !argo submit masters-workflow.yaml -o name
workflow = workflow[0]

In [ ]:
while "workflow" not in db.get_data(redis, workflow):
    continue

time.sleep(1)

raw_data = db.get_data(redis, workflow)
theory_data = process.process_theory_data(raw_data)

In [ ]:
while True:
    try:
        ax.clear()
        raw_data = db.get_data(redis, workflow)
        processed_data = process.process(raw_data)
        plot.plot(ax, raw_data, theory_data, processed_data)
        fig.canvas.draw()
        time.sleep(1)
    except KeyboardInterrupt:
        break